In [2]:

import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import clustering

In [3]:
PATH_TO_DATA = "../data"
# DATA_FILE = "/ppv_state-district-geo-dataAug2021.csv"
DATA_FILE = "/ppv_skp-geo-pop-data.csv"
data_src = PATH_TO_DATA + DATA_FILE
df = pd.read_csv(data_src)

COORD_COLS = ['latitude', 'longitude']

In [4]:
np.random.seed(0)

In [6]:
gombak = df[df.district == 'Gombak']
gombak[COORD_COLS].values

array([[  3.2360267 , 101.6296346 ],
       [  3.2919717 , 101.7309713 ],
       [  3.2426173 , 101.6464691 ],
       [  3.2196265 , 101.5831991 ],
       [  3.1749588 , 101.7487479 ],
       [  3.24627404, 101.72260143],
       [  3.3201676 , 101.4736959 ],
       [  3.2178664 , 101.7142398 ],
       [  3.2594189 , 101.55195   ],
       [  3.3147693 , 101.5958835 ],
       [  3.245237  , 101.6738341 ],
       [  3.2192844 , 101.5779949 ],
       [  3.224222  , 101.6229275 ],
       [  3.28321805, 101.53893315],
       [  3.22103425, 101.59345045],
       [  3.30017675, 101.64358485],
       [  3.2511277 , 101.7279981 ]])

In [9]:
n_split = 5
shuffled = gombak.sample(frac=1)
splits = np.array_split(shuffled, n_split)

In [12]:
splits[0]

,state,district,ppv_name,latitude,longitude,pop_density,pop_growth
5,Selangor,Gombak,Klinik Kesihatan Batu 8,3.246274,101.722601,1270,0.017
13,Selangor,Gombak,"Le Garden Pavilion, Gamuda Gardens, Rawang, Go...",3.283218,101.538933,1270,0.017
7,Selangor,Gombak,Klinik Kesihatan Gombak Setia,3.217866,101.714240,1270,0.017
4,Selangor,Gombak,Klinik Kesihatan AU2,3.174959,101.748748,1270,0.017


In [16]:
centroids = []
split = splits[0]
split_coords = split[COORD_COLS].values
centroid = split_coords.mean(axis=0) # Weightless centroid
centroids.append(centroid)

In [34]:
def euclidean_metric(x: np.ndarray, y: np.ndarray):
    distance = np.sqrt(((x-y)**2).sum())
    return distance

def lp_metric(x: np.ndarray, y: np.ndarray, p: float):
    if p <= 0:
        raise ValueError("Expect 'p' to be a positive real number.")
    distance = ((abs(x-y)**p).sum())**(1./p)
    return distance

In [35]:
print(lp_metric(np.array([1, 1]), np.array([2, 2]), p=1))
euclidean_metric(np.array([1, 1]), np.array([2, 2]))

2.0


1.4142135623730951

In [48]:
import itertools
from scipy.spatial.distance import cdist

In [110]:
def spherical_distance(x, y):
    distance = np.arccos(np.sin(x[0])*np.sin(y[0]) + np.cos(x[0])*np.cos(y[0])*np.cos(y[1] - x[1]))
    return distance

In [111]:
split_coords

array([[  3.24627404, 101.72260143],
       [  3.28321805, 101.53893315],
       [  3.2178664 , 101.7142398 ],
       [  3.1749588 , 101.7487479 ]])

In [150]:
radii = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0, 1, 10, 100, 1000]
for r in radii:
    res = clustering.two_nearest_circle_point_centroid(split_coords[1], split_coords[3], r, r+0.188)
    print(f"r: {r}, res: {res}")

r: 1e-06, res: [  3.60662046 111.903096  ]
r: 1e-05, res: [  3.60662046 111.903096  ]
r: 0.0001, res: [  3.60662046 111.903096  ]
r: 0.001, res: [  3.60662046 111.903096  ]
r: 0.01, res: [  3.60662046 111.903096  ]
r: 0.1, res: [  3.60662046 111.903096  ]
r: 0, res: [  3.60662046 111.903096  ]
r: 1, res: [  3.60662046 111.903096  ]
r: 10, res: [  3.60662046 111.903096  ]
r: 100, res: [  3.60662046 111.903096  ]
r: 1000, res: [  3.60662046 111.903096  ]


In [113]:
euclidean_metric(split_coords[0], split_coords[1])

0.18734699159893353

In [49]:
def closest_node(node, nodes):
    return nodes[cdist([node], nodes).argmin()]

In [70]:
node = [3.24627404, 101.72260143]
nodes = split_coords

cdist([node], nodes).nonzero()

(array([0, 0, 0, 0]), array([0, 1, 2, 3]))

In [52]:
split_coords

array([[  3.24627404, 101.72260143],
       [  3.28321805, 101.53893315],
       [  3.2178664 , 101.7142398 ],
       [  3.1749588 , 101.7487479 ]])

In [88]:

# for node in split_coords:
#     temp_split_coords = np.delete(split_coords, np.argwhere(split_coords==node))
#     temp_split_coords = temp_split_coords.reshape(len(split_coords)-1, -1)
#     print(temp_split_coords)
#     print(closest_node(node, temp_split_coords), end='\n\n')

In [109]:
count = 0
for pair in itertools.product(split_coords, repeat=2):
    x, y = pair
    if np.array_equal(x, y):
        continue
    c = clustering.two_nearest_circle_point_centroid(x, y, 5, 5)
    print(x, y)
    print(c)
    print()
    count += 1
    if count == len(split_coords):
        break

[  3.24627404 101.72260143] [  3.28321805 101.53893315]
[  4.88788307 152.492068  ]

[  3.24627404 101.72260143] [  3.2178664 101.7142398]
[  4.85520724 152.57972133]

[  3.24627404 101.72260143] [  3.1749588 101.7487479]
[  4.83375344 152.59697538]

[  3.28321805 101.53893315] [  3.24627404 101.72260143]
[  4.90635507 152.40023386]

